In [ ]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available!")
else:
    print("GPU NOT available.")

GPU is available!


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import zipfile
import os

# فك الضغط للملف الأول في مجلد "new"
with zipfile.ZipFile("/content/new.zip", 'r') as zip_ref:
    zip_ref.extractall("new")

# فك الضغط للملف الثاني في مجلد "test"
with zipfile.ZipFile("/content/test.zip", 'r') as zip_ref:
    zip_ref.extractall("test")

# طباعة محتويات المجلدين
print("new contents:", os.listdir("new"))
print("test contents:", os.listdir("test"))


new contents: ['new']
test contents: ['test']


In [ ]:
path1 = "new"
path2 = "test"

# طباعة محتوى المجلدات بعد فك الضغط
print(f"{path1} contents:", os.listdir(path1))
print(f"{path2} contents:", os.listdir(path2))

new contents: ['new']
test contents: ['test']


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os

In [ ]:
def setup_data_paths():
    # Your existing directories
    test_dir = "/content/organized_dataset/test"
    val_dir = "/content/organized_dataset/validation"
    train_dir = "/content/organized_dataset/train"

    return train_dir, val_dir, test_dir

# Setup paths for your existing structure
train_dir, val_dir, test_dir = setup_data_paths()

In [ ]:
def check_data_availability():
    """Check if data is available in your existing directories"""
    print("\nChecking data availability in your folders...")

    dirs_to_check = [
        os.path.join(train_dir, "brain_glioma"),      # new/brain_glioma
        os.path.join(train_dir, "brain_menin"),  # new/brain_menin
        os.path.join(val_dir, "brain_glioma"),        # val/brain_glioma
        os.path.join(val_dir, "brain_menin"),    # val/brain_menin
        os.path.join(test_dir, "brain_glioma"),       # test/brain_glioma
        os.path.join(test_dir, "brain_menin")    # test/brain_menin
    ]

    expected_counts = [1300, 61, 400, 18, 300, 15]  # Expected image counts
    total_images = 0

    for i, dir_path in enumerate(dirs_to_check):
        if os.path.exists(dir_path):
            num_files = len([f for f in os.listdir(dir_path)
                           if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
            expected = expected_counts[i]
            status = "✅" if num_files > 0 else "⚠️"
            print(f"{status} {dir_path}: {num_files} images (expected: {expected})")
            total_images += num_files

            if num_files == 0:
                print(f"   ❌ WARNING: No images found in {dir_path}")
        else:
            print(f"❌ Directory not found: {dir_path}")
            print(f"   💡 Make sure you have subfolders: brain_glioma and brain_meningioma")

    print(f"\nTotal images found: {total_images}")
    print(f"Expected total: {sum(expected_counts)}")

    if total_images > 0:
        print("✅ Data is available! Ready to proceed.")
    else:
        print("⚠️  No images found. Please check your folder structure.")

    return total_images > 0

# Check if data is available
data_available = check_data_availability()


Checking data availability in your folders...
❌ Directory not found: /content/organized_dataset/train/brain_glioma
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma
❌ Directory not found: /content/organized_dataset/train/brain_menin
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma
❌ Directory not found: /content/organized_dataset/validation/brain_glioma
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma
❌ Directory not found: /content/organized_dataset/validation/brain_menin
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma
❌ Directory not found: /content/organized_dataset/test/brain_glioma
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma
❌ Directory not found: /content/organized_dataset/test/brain_menin
   💡 Make sure you have subfolders: brain_glioma and brain_meningioma

Total images found: 0
Expected total: 2094
⚠️  No images found. Please check your folder structure.


In [ ]:
def create_model(img_height=224, img_width=224, num_classes=3):
    """Create lightweight transfer learning model using MobileNetV2"""
    # Load pre-trained MobileNetV2 (much smaller than ResNet50)
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(img_height, img_width, 3),
        alpha=1.0  # Width multiplier, use 0.75 or 0.5 for even smaller model
    )

    # Freeze base model layers initially
    base_model.trainable = False

    # Add lightweight custom classification head
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)  # Smaller dense layer
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)   # Even smaller
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    print(f"Model created with {model.count_params():,} parameters")
    return model

# Create the model
model = create_model()
print("\nModel Summary:")
model.summary()

Model created with 2,435,523 parameters

Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,435,523 (9.29 MB)

 Trainable params: 174,979 (683.51 KB)

 Non-trainable params: 2,260,544 (8.62 MB)

In [ ]:
def prepare_data_generators(batch_size=16):
    """Prepare data generators with augmentation"""

    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,
        zoom_range=0.15,
        brightness_range=[0.9, 1.1],
        fill_mode='nearest'
    )

    # Validation and test data generators (only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator, test_generator

# Only prepare generators if data is available
if data_available:
    train_gen, val_gen, test_gen = prepare_data_generators(batch_size=16)
    print("Data generators created successfully!")
    print(f"Training samples: {train_gen.samples}")
    print(f"Validation samples: {val_gen.samples}")
    print(f"Test samples: {test_gen.samples}")
    print(f"Class indices: {train_gen.class_indices}")
else:
    print("⚠️ Data not available. Please place images in the folders first.")

⚠️ Data not available. Please place images in the folders first.


In [ ]:
def calculate_class_weights():
    """Calculate class weights for imbalanced dataset"""
    # Based on your distribution
    total_glioma = 1300 + 400 + 300  # 2000
    total_menin = 61 + 18 + 15   # 94
    total_samples = total_glioma + total_menin

    weight_glioma = total_samples / (2 * total_glioma)
    weight_menin = total_samples / (2 * total_menin)

    class_weights = {0: weight_glioma, 1: weight_menin}
    print(f"Class weights calculated:")
    print(f"  - Glioma (class 0): {weight_glioma:.3f}")
    print(f"  - Menin (class 1): {weight_menin:.3f}")
    return class_weights

# Calculate class weights
class_weights = calculate_class_weights()

Class weights calculated:
  - Glioma (class 0): 0.523
  - Menin (class 1): 11.138


In [ ]:
def get_callbacks():
    """Define training callbacks"""
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=8,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=4,
            min_lr=1e-7,
            verbose=1
        )
    ]
    return callbacks

# Setup callbacks
callbacks = get_callbacks()
print("Training callbacks configured:")
print("- Early stopping (patience=8)")
print("- Learning rate reduction (patience=4)")

Training callbacks configured:
- Early stopping (patience=8)
- Learning rate reduction (patience=4)


In [ ]:
if data_available:
    print("Starting Phase 1: Training with frozen base model...")

    # Initial training with frozen base
    history_phase1 = model.fit(
        train_gen,
        epochs=15,
        validation_data=val_gen,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )

    print("Phase 1 completed!")
else:
    print("⚠️ Skipping training - data not available")

⚠️ Skipping training - data not available


In [ ]:
if data_available:
    print("Starting Phase 2: Fine-tuning...")

    # Unfreeze last layers for fine-tuning
    base_model = model.layers[1]
    base_model.trainable = True

    # Freeze earlier layers, only train last 20 layers
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    # Use lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    # Continue training
    history_phase2 = model.fit(
        train_gen,
        epochs=15,
        validation_data=val_gen,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )

    # Combine histories
    history = {
        'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
        'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
        'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
        'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
        'precision': history_phase1.history['precision'] + history_phase2.history['precision'],
        'val_precision': history_phase1.history['val_precision'] + history_phase2.history['val_precision'],
        'recall': history_phase1.history['recall'] + history_phase2.history['recall'],
        'val_recall': history_phase1.history['val_recall'] + history_phase2.history['val_recall']
    }

    print("Phase 2 completed! Training finished.")
else:
    print("⚠️ Skipping fine-tuning - data not available")

In [ ]:
def plot_training_history(history):
    """Plot training history"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    # Accuracy
    axes[0, 0].plot(history['accuracy'], label='Training', color='blue')
    axes[0, 0].plot(history['val_accuracy'], label='Validation', color='red')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss
    axes[0, 1].plot(history['loss'], label='Training', color='blue')
    axes[0, 1].plot(history['val_loss'], label='Validation', color='red')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision
    axes[1, 0].plot(history['precision'], label='Training', color='blue')
    axes[1, 0].plot(history['val_precision'], label='Validation', color='red')
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Recall
    axes[1, 1].plot(history['recall'], label='Training', color='blue')
    axes[1, 1].plot(history['val_recall'], label='Validation', color='red')
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Plot training history if available
if data_available and 'history' in locals():
    plot_training_history(history)
else:
    print("⚠️ Training history not available - train the model first")

In [ ]:
def evaluate_model(model, test_generator):
    """Evaluate model on test set and plot confusion matrix"""
    # Predict class probabilities
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes

    # Get class names from generator
    class_names = list(test_generator.class_indices.keys())

    # Safety check for mismatch between predicted classes and class names
    unique_classes = np.unique(true_classes)
    if len(class_names) != len(unique_classes):
        print(f"⚠️ Mismatch detected:")
        print(f" - test_generator.class_indices has {len(class_names)} classes: {class_names}")
        print(f" - test set has {len(unique_classes)} classes: {unique_classes.tolist()}")
        print("ℹ️ Adjusting class_names to match the true classes in test set...")
        class_names = class_names[:len(unique_classes)]

    # Classification report
    print("\n📋 Classification Report:")
    print(classification_report(true_classes, predicted_classes, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # Evaluate overall metrics
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
    print(f"\n✅ Test Set Evaluation:")
    print(f"Loss     : {test_loss:.4f}")
    print(f"Accuracy : {test_accuracy:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall   : {test_recall:.4f}")

    # F1 Score
    if test_precision + test_recall > 0:
        f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
    else:
        f1_score = 0.0
    print(f"F1-Score : {f1_score:.4f}")

In [ ]:
if data_available and 'test_gen' in locals():
    evaluate_model(model, test_gen)
else:
    print("⚠️ Cannot evaluate - data not available or model not trained")

In [ ]:
model.save("brain_tumor_model_final.h5")
print("✅ Model saved as 'brain_tumor_model_final.h5'")

In [ ]:
# Quick overfitting check
final_train_acc = history['accuracy'][-1]
final_val_acc = history['val_accuracy'][-1]
gap = final_train_acc - final_val_acc

print(f"Final Train Accuracy: {final_train_acc:.4f}")
print(f"Final Val Accuracy:   {final_val_acc:.4f}")
print(f"Accuracy Gap:         {gap:.4f}")

if gap > 0.1:
    print("⚠️ Possible overfitting detected.")
else:
    print("✅ No strong sign of overfitting.")